In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
q_a_data = pd.read_csv('/content/drive/MyDrive/Books/Ai Writting/Conversation/final_Chat_Data_GBC.csv')
q_a_data = q_a_data.drop("sentiment",axis=1)
persona_data = pd.read_csv('/content/drive/MyDrive/Books/Ai Writting/Conversation/personality.csv')
persona_data = persona_data.drop("Unnamed: 0",axis = 1)

In [10]:
q_a_data.head()

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [11]:
persona_data.head()

,Persona,chat
0,i like to remodel homes. i like to go hunting...,"hi , how are you doing ? i am getting ready to..."
1,my mom is my best friend. i have four sisters...,"hi , how are you doing today ?\ni am spending ..."
2,i had a gig at local theater last night. i wo...,"we all live in a yellow submarine , a yellow s..."
3,i am very athletic. i wear contacts. i have b...,hi ! i work as a gourmet cook .\ni do not like...
4,i am primarily a meat eater. i am a guitar pl...,how are you doing today\nwhat do you do for ca...


In [4]:
data1 = q_a_data
data2 = persona_data

In [21]:
data2.isnull().sum()

Persona    0
chat       0
dtype: int64

In [5]:
import pandas as pd

# Select relevant columns from the datasets
data1_selected = data1[["question", "answer"]]
data2_selected = data2[["Persona", "chat"]]

# Join question and persona, answer and chat into single columns
data1_selected["text"] = data1_selected["question"]
data1_selected.loc[len(data1_selected)-1, "text"] += " " + data2_selected["Persona"].values[0]
data1_selected["response"] = data1_selected["answer"]
data1_selected.loc[len(data1_selected)-1, "response"] += " " + data2_selected["chat"].values[0]

# Drop unnecessary columns
data1_selected = data1_selected[["text", "response"]]

# Concatenate the datasets
combined_data = pd.concat([data1_selected, data2_selected.iloc[1:]])

# Reset the index of the combined dataset
combined_data.reset_index(drop=True, inplace=True)

# Replace NaN values with empty string
combined_data.fillna("", inplace=True)

combined_data['User'] = combined_data['text'] + combined_data['Persona']
combined_data['Reply'] = combined_data['response'] + combined_data['chat']

In [6]:
cols_drop = ["text" ,"response", "Persona","chat"]
combined_data = combined_data.drop(cols_drop,axis=1)
combined_data.tail()

,User,Reply
12658,my favorite food is fried chicken. i have a t...,"good evening , i have been having a terrible t..."
12659,i love fast food. i am a stay at home mom. i ...,hi there how is work\ni do not work these days...
12660,my family and i go camping every month. i am ...,"hello , i have a daughter who is in kindergart..."
12661,i have red hair. i work at a retail store. i ...,hi . how are you doing ?\ni am doing well . ju...
12662,i m applying for publishing jobs. the only au...,hello ! how are you today ?\nhello i am good ....


In [7]:
combined_data.to_csv('Final_OUT_Conv_Data.csv')

#Pre Processing Dataset

**Text Cleaning:** Remove any unnecessary characters, symbols, or special characters that do not contribute to the meaning of the text. This includes removing HTML tags, URLs, or any noise specific to your dataset.

**Lowercasing:** Convert the text to lowercase to ensure consistency and avoid treating words with different casing as different entities.

**Tokenization:** Split the text into individual tokens, which can be words or subword units. Tokenization helps the model understand the structure of the text and build its vocabulary.

**Special Tokens:** Add special tokens to denote the beginning and end of sentences or prompts, as well as special tokens for padding or masking if necessary.

**Sequence Length Limit:** Set a maximum sequence length for the input data. If a sequence exceeds this limit, it needs to be truncated or split into smaller segments to fit within the model's memory constraints.

**Handling Long Texts:** For long texts that cannot fit into a single sequence, you can use techniques like sliding window or recursive encoding to process the text in smaller chunks.

**Data Encoding:** Map the tokens to their corresponding integer IDs according to the model's vocabulary. This step transforms the text data into a numerical representation that can be processed by the model.

**Data Formatting:** Organize the encoded data into input-output pairs suitable for the model. For example, for autoregressive models like GPT, the input sequence can be followed by the target sequence shifted by one position.

**Masking:** Apply attention masks or padding masks to indicate which tokens are valid input and which should be ignored during training. This helps the model focus on relevant tokens and handle varying sequence lengths.

**Data Batching:** Group the input-output pairs into batches for more efficient training. Batching allows parallel processing and improves training speed.

In [8]:
!pip install transformers tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.2 MB/s eta 0:00:00


In [9]:
import re
import torch
from transformers import GPT2Tokenizer

In [10]:
data = combined_data

##Cleaning Text


1. Removing leading and trailing spaces using the strip method.

2. Removing HTML tags using a regular expression pattern (<.*?>) and the re.sub function.

3. Removing URLs using a regular expression pattern (http\S+|www\S+|https\S+) and the re.sub function.

4. Removing special characters or symbols using a regular expression pattern ([^a-zA-Z0-9\s]) and the re.sub function.

5. Normalizing whitespace using a regular expression pattern (\s+) and the re.sub function.

6. Replacing multiple consecutive punctuation marks (e.g., "!!!" or "??") with a single one using a regular expression pattern (([!?.]){2,}) and the re.sub function.

7. Replacing numbers with a special token (e.g., "NUM") using a regular expression pattern (\d+) and the re.sub function.

8. Lowering the text to small letters






In [11]:
import re

def clean_text(text):
    # Remove leading/trailing spaces
    text = text.strip()
    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Remove special characters or symbols
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)
    # Replace multiple consecutive punctuation marks with a single one
    text = re.sub(r"([!?.]){2,}", r"\1", text)
    # Replace numbers with a special token
    text = re.sub(r"\d+", "NUM", text)

    return text

# Apply text cleaning to "User" and "Reply" columns
data["User"] = data["User"].apply(clean_text)
data["Reply"] = data["Reply"].apply(clean_text)

# Step 2: Lowercasing
data["User"] = data["User"].str.lower()
data["Reply"] = data["Reply"].str.lower()


##Tokenization

In [12]:
# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
data["User_tokens"] = data["User"].apply(tokenizer.tokenize)
data["Reply_tokens"] = data["Reply"].apply(tokenizer.tokenize)

In [13]:
# Step 4: Special Tokens
special_tokens = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
}
tokenizer.add_special_tokens(special_tokens)

4

In [14]:
# Step 6: Data Encoding
data["User_ids"] = data["User_tokens"].apply(tokenizer.convert_tokens_to_ids)
data["Reply_ids"] = data["Reply_tokens"].apply(tokenizer.convert_tokens_to_ids)

In [15]:
# Step 7: Data Formatting
formatted_data = [(user_ids, reply_ids) for user_ids, reply_ids in zip(data["User_ids"], data["Reply_ids"])]

In [16]:
# Step 8: Masking
attention_mask = [[1] * len(user_ids) for user_ids in data["User_ids"]]

In [17]:
# Step 9: Data Batching
batch_size = 16  # Set the desired batch size

In [18]:
# Pad the sequences to a fixed length
padded_input_ids = torch.nn.utils.rnn.pad_sequence([torch.tensor(ids) for ids, _ in formatted_data], batch_first=True)
padded_attention_mask = torch.nn.utils.rnn.pad_sequence([torch.tensor(mask) for _, mask in zip(data["User_ids"], attention_mask)], batch_first=True)

In [19]:
# Convert the padded sequences and attention masks into PyTorch tensors
input_ids = torch.tensor(padded_input_ids)
attention_mask = torch.tensor(padded_attention_mask)

# Create data batches
data_batches = torch.utils.data.TensorDataset(input_ids, attention_mask)
data_loader = torch.utils.data.DataLoader(data_batches, batch_size=batch_size, shuffle=True)

<ipython-input-19-c5607bf1755d>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(padded_input_ids)
<ipython-input-19-c5607bf1755d>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(padded_attention_mask)


Summary Code

In [85]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer
from torch.nn.utils.rnn import pad_sequence



# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Step 1: Text Cleaning
# Step 2: Lowercasing
# Step 3: Tokenization
# Step 4: Special Tokens
# Step 5: Sequence Length Limit
# Step 6: Data Encoding
formatted_data = []
attention_mask = []

for user_text, reply_text in zip(data["User"], data["Reply"]):
    # Text Cleaning
    user_text = clean_text(user_text)
    reply_text = clean_text(reply_text)

    # Lowercasing
    user_text = user_text.lower()
    reply_text = reply_text.lower()

    # Tokenization
    user_tokens = tokenizer.tokenize(user_text)
    reply_tokens = tokenizer.tokenize(reply_text)

    # Special Tokens
    user_tokens = [tokenizer.bos_token] + user_tokens + [tokenizer.eos_token]
    reply_tokens = [tokenizer.bos_token] + reply_tokens + [tokenizer.eos_token]

    # Sequence Length Limit
    max_sequence_length = 128  # Set the maximum sequence length
    user_tokens = user_tokens[:max_sequence_length]
    reply_tokens = reply_tokens[:max_sequence_length]

    # Data Encoding
    user_ids = tokenizer.convert_tokens_to_ids(user_tokens)
    reply_ids = tokenizer.convert_tokens_to_ids(reply_tokens)

    formatted_data.append((user_ids, reply_ids))
    attention_mask.append([1] * len(user_ids))

# # Step 7: Data Formatting
# input_ids = pad_sequence([torch.tensor(user_ids) for user_ids, _ in formatted_data], batch_first=True)
# target_ids = pad_sequence([torch.tensor(reply_ids) for _, reply_ids in formatted_data], batch_first=True)
# attention_mask = pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True)

# Step 7: Data Formatting
input_ids = pad_sequence([torch.tensor(user_ids) for user_ids, _ in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
target_ids = pad_sequence([torch.tensor(reply_ids) for _, reply_ids in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
attention_mask = pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True)


# Step 8: Masking
attention_mask = attention_mask.bool()

# Step 9: Data Batching
batch_size = 16  # Set the desired batch size

# Create data batches
data_batches = torch.utils.data.TensorDataset(input_ids, attention_mask, target_ids)
data_loader = torch.utils.data.DataLoader(data_batches, batch_size=batch_size, shuffle=True)


#Model Building

In [86]:
from transformers import GPT2LMHeadModel

In [87]:
# Load the pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set the model to the desired device
model = model.to(device)

In [88]:
# Set hyperparameters
learning_rate = 1e-4
num_epochs = 3

In [82]:
# Set optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    total_batches = 0

    for batch_idx, (input_ids, attention_mask, target_ids) in enumerate(data_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        target_ids = target_ids.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

    # Calculate average loss for the epoch
    average_loss = total_loss / total_batches

    # Update learning rate
    scheduler.step()

    # Print progress
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {average_loss}")

# Save the trained model
model.save_pretrained("trained_model")
tokenizer.save_pretrained("trained_model")

In [27]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.nn.utils.rnn import pad_sequence

average_loss_p = []

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Step 1: Text Cleaning
# Step 2: Lowercasing
# Step 3: Tokenization
# Step 4: Special Tokens
# Step 5: Sequence Length Limit
# Step 6: Data Encoding
formatted_data = []
attention_mask = []

for user_text, reply_text in zip(data["User"], data["Reply"]):
    # Text Cleaning
    user_text = clean_text(user_text)
    reply_text = clean_text(reply_text)

    # Lowercasing
    user_text = user_text.lower()
    reply_text = reply_text.lower()

    # Tokenization
    user_tokens = tokenizer.tokenize(user_text)
    reply_tokens = tokenizer.tokenize(reply_text)

    # Special Tokens
    user_tokens = [tokenizer.bos_token] + user_tokens + [tokenizer.eos_token]
    reply_tokens = [tokenizer.bos_token] + reply_tokens + [tokenizer.eos_token]

    # Sequence Length Limit
    max_sequence_length = 60 # Set the maximum sequence length
    user_tokens = user_tokens[:max_sequence_length]
    reply_tokens = reply_tokens[:max_sequence_length]

    # Data Encoding
    user_ids = tokenizer.convert_tokens_to_ids(user_tokens)
    reply_ids = tokenizer.convert_tokens_to_ids(reply_tokens)

    formatted_data.append((user_ids, reply_ids))
    attention_mask.append([1] * len(user_ids))

# Step 7: Data Formatting
input_ids = pad_sequence([torch.tensor(user_ids) for user_ids, _ in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
target_ids = pad_sequence([torch.tensor(reply_ids) for _, reply_ids in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
attention_mask = pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True)

# Step 8: Masking
attention_mask = attention_mask.bool()

# Filter out incomplete batches
complete_batches = [idx for idx, (input_batch, target_batch) in enumerate(zip(input_ids, target_ids)) if input_batch.size(0) == target_batch.size(0)]
input_ids = input_ids[complete_batches]
target_ids = target_ids[complete_batches]
attention_mask = attention_mask[complete_batches]

# Step 9: Data Batching
batch_size = 64  # Set the desired batch size

if len(input_ids) > 0:
    # Create data batches
    data_batches = torch.utils.data.TensorDataset(input_ids, attention_mask, target_ids)
    data_loader = torch.utils.data.DataLoader(data_batches, batch_size=batch_size, shuffle=True)

    # Load the pre-trained GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    # Set the device for training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Set the model to the desired device
    model = model.to(device)
    # Set hyperparameters
    learning_rate = 1e-4
    num_epochs = 50
    # Set optimizer and learning rate scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

   # Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    total_batches = 0

    for batch_idx, (input_ids, attention_mask, target_ids) in enumerate(data_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        target_ids = target_ids.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

        # Print progress
        print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx+1}/{len(data_loader)} | Loss: {loss.item()}")

    # Calculate average loss for the epoch
    average_loss = total_loss / total_batches
    average_loss_p.append(average_loss)
    # Update learning rate
    scheduler.step()

    # Print progress
    print(f"Epoch {epoch+1}/{num_epochs} | Average Loss: {average_loss}")


    # Save the trained model
    model.save_pretrained("trained_model")
    tokenizer.save_pretrained("trained_model")
else:
    print("No complete batches remaining after filtering.")


Streaming output truncated to the last 5000 lines.
Epoch 25/50 | Batch 176/198 | Loss: 1.9287289381027222
Epoch 25/50 | Batch 177/198 | Loss: 1.6140964031219482
Epoch 25/50 | Batch 178/198 | Loss: 2.109032154083252
Epoch 25/50 | Batch 179/198 | Loss: 1.8018094301223755
Epoch 25/50 | Batch 180/198 | Loss: 2.0459835529327393
Epoch 25/50 | Batch 181/198 | Loss: 1.8262004852294922
Epoch 25/50 | Batch 182/198 | Loss: 1.6854921579360962
Epoch 25/50 | Batch 183/198 | Loss: 1.6793545484542847
Epoch 25/50 | Batch 184/198 | Loss: 1.9061262607574463
Epoch 25/50 | Batch 185/198 | Loss: 2.0613205432891846
Epoch 25/50 | Batch 186/198 | Loss: 1.9465641975402832
Epoch 25/50 | Batch 187/198 | Loss: 1.816077709197998
Epoch 25/50 | Batch 188/198 | Loss: 1.9491405487060547
Epoch 25/50 | Batch 189/198 | Loss: 1.8865643739700317
Epoch 25/50 | Batch 190/198 | Loss: 1.8258165121078491
Epoch 25/50 | Batch 191/198 | Loss: 1.6907621622085571
Epoch 25/50 | Batch 192/198 | Loss: 1.7629475593566895
Epoch 25/50 | Ba

In [28]:
path = '/content/drive/MyDrive/Books/trained_model'

# Save the trained model
model.save_pretrained(path)

# Save the tokenizer
tokenizer.save_pretrained("path")

# Save the model configuration
model.config.save_pretrained(path)

# Save other relevant components
torch.save(optimizer.state_dict(),path + "/optimizer.pth")
torch.save(scheduler.state_dict(), path+"/scheduler.pth")

In [35]:
for i in range(0, len(average_loss_p)):
    loss = str(average_loss_p[i])
    print("For Epoch {} Avg loss is {}".format(i + 1, loss))

For Epoch 1 Avg loss is 4.4664851619739725
For Epoch 2 Avg loss is 4.280565104099235
For Epoch 3 Avg loss is 4.23955447264392
For Epoch 4 Avg loss is 4.198595897115842
For Epoch 5 Avg loss is 4.14556430445777
For Epoch 6 Avg loss is 4.075488082086197
For Epoch 7 Avg loss is 3.984121063742975
For Epoch 8 Avg loss is 3.870536218989979
For Epoch 9 Avg loss is 3.735497242272502
For Epoch 10 Avg loss is 3.584352909916579
For Epoch 11 Avg loss is 3.422223590841197
For Epoch 12 Avg loss is 3.25636736672334
For Epoch 13 Avg loss is 3.0883133893061165
For Epoch 14 Avg loss is 2.9247889928143436
For Epoch 15 Avg loss is 2.766388942496945
For Epoch 16 Avg loss is 2.6202012408863413
For Epoch 17 Avg loss is 2.4884209109075144
For Epoch 18 Avg loss is 2.366399875794998
For Epoch 19 Avg loss is 2.259764075279236
For Epoch 20 Avg loss is 2.1642836675499426
For Epoch 21 Avg loss is 2.078123009566105
For Epoch 22 Avg loss is 2.001676698525747
For Epoch 23 Avg loss is 1.9332367747721046
For Epoch 24 Avg

In [47]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "thanks"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: thanks. you always end stories


In [52]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "chat with me"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: chat with me for doing!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [54]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "do you have any sentiments to find"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: do you have any sentiments to find!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [57]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "describe a story for me on any new topic"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: describe a story for me on any new topic you could too to the the the the a the the gets is no to no that the no on be no to the do to women doctor on what to to to wear to! for to draw a
